# Предобработка данных

- Добавим столбец `target_year`, который ставит в соответствие объект и таргет. В частности, для периода месяцев *ноябрь-декабрь* `target_year = year + 1`, т.к. сбор урожая кукурузы происходит в *сентябре-октябре*.
- Удаляем данные *первого года январь-октябрь* и *последнего года ноябрь-декабрь*
- Удаляем лишние признаки: координаты, пути к изображениям, максимальная и минимальные температуры
- Соединяем признаки с таргетами по (`target_year`, `fips`) и (`year`, `target`) соответственно
- Обработка `NaN` значений
- Разделяем на `train`, `test`. В качестве `test` возьмем последний год

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

PATH_INTERIM = Path("../../data/interim")
PATH_PROCESSED = Path("../../data/processed")

In [2]:
X = pd.read_csv(PATH_INTERIM / "X.csv")
y = pd.read_csv(PATH_INTERIM / "y.csv")

## Добавим столбец `target_year`

In [3]:
X["target_year"] = np.where(X["month"] >= 11, X["year"] + 1, X["year"])

## Удаляем данные *первого года январь-октябрь* и *последнего года ноябрь-декабрь*

In [4]:
min_year, max_year = X["year"].min(), X["year"].max()
X = X[
    ~(
        ((X["year"] == min_year) & (X["month"] < 11))
        | ((X["year"] == max_year) & (X["month"] > 10))
    )
]

## Удаляем лишние признаки

In [5]:
X.drop(
    [
        "lat_lower_left",
        "lon_lower_left",
        "lat_upper_right",
        "lon_upper_right",
        "temperature_max",
        "temperature_avg",
    ],
    axis=1,
    inplace=True,
)

In [6]:
display(X.shape)
X.head(5)

(69000, 19)

,year,fips,month,day,temperature_min,precipitation,humidity_relative,wind_gust,wind_speed,wind_u_component,wind_v_component,solar_radiation_downward,vapor_pressure_deficit,skin_reservoir_content,soil_water_vol_layer1,soil_water_vol_layer2,soil_water_vol_layer3,images,target_year
20,2017,17001,11,1,273.936458,0.187208,71.783333,7.191125,3.372792,-2.079917,2.556042,444.233333,0.222625,4.768372e-07,0.310633,0.317448,0.258163,images/17001-2017-11-01,2018
21,2017,17001,11,15,281.599000,5.130583,66.950000,8.537250,4.711750,2.010292,0.684500,2356.704167,0.475833,2.998769e-04,0.411875,0.311898,0.264373,images/17001-2017-11-15,2018
22,2017,17001,12,1,270.082875,0.000000,54.183333,3.181625,2.111375,0.135792,1.666417,2724.837500,0.460125,5.270541e-06,0.313241,0.322834,0.293687,images/17001-2017-12-01,2018
23,2017,17001,12,15,268.693333,0.196958,69.087500,5.463792,3.034458,2.569542,0.937000,1957.187500,0.215583,3.866851e-07,0.308099,0.315572,0.295845,images/17001-2017-12-15,2018
44,2017,17003,11,1,278.042667,18.158111,90.355556,4.224000,2.226778,-1.714222,0.091889,61.733333,0.099333,3.265850e-04,0.382662,0.323411,0.240666,images/17003-2017-11-01,2018


## Соединяем признаки с таргетами

In [7]:
y["year"] = y["year"].astype(X["target_year"].dtype)
data = pd.merge(
    X.drop("year", axis=1),
    y,
    how="left",
    left_on=["target_year", "fips"],
    right_on=["year", "fips"],
)
# target_year не имеет NaN, year может иметь
data.drop("year", axis=1, inplace=True)
data.rename({"target_year": "year"}, inplace=True, axis=1)
display(data.shape)
data.head()

(69000, 19)

,fips,month,day,temperature_min,precipitation,humidity_relative,wind_gust,wind_speed,wind_u_component,wind_v_component,solar_radiation_downward,vapor_pressure_deficit,skin_reservoir_content,soil_water_vol_layer1,soil_water_vol_layer2,soil_water_vol_layer3,images,year,yield_bu_per_acre
0,17001,11,1,273.936458,0.187208,71.783333,7.191125,3.372792,-2.079917,2.556042,444.233333,0.222625,4.768372e-07,0.310633,0.317448,0.258163,images/17001-2017-11-01,2018,197.8
1,17001,11,15,281.599000,5.130583,66.950000,8.537250,4.711750,2.010292,0.684500,2356.704167,0.475833,2.998769e-04,0.411875,0.311898,0.264373,images/17001-2017-11-15,2018,197.8
2,17001,12,1,270.082875,0.000000,54.183333,3.181625,2.111375,0.135792,1.666417,2724.837500,0.460125,5.270541e-06,0.313241,0.322834,0.293687,images/17001-2017-12-01,2018,197.8
3,17001,12,15,268.693333,0.196958,69.087500,5.463792,3.034458,2.569542,0.937000,1957.187500,0.215583,3.866851e-07,0.308099,0.315572,0.295845,images/17001-2017-12-15,2018,197.8
4,17003,11,1,278.042667,18.158111,90.355556,4.224000,2.226778,-1.714222,0.091889,61.733333,0.099333,3.265850e-04,0.382662,0.323411,0.240666,images/17003-2017-11-01,2018,188.4


## Сортировка

Приводим строки в правильный порядок. Т.к. считаем урожайность на данных ноябрь-август, то первыми должны идти данные за ноябрь-декабрь предыдущего года (в текущем `data` все ноябри-декабри относятся к "будущему году". Напр., у фактического ноября 2017-ого в `year` будет стоять 2018. Так сделано для удобства)

In [8]:
data["month_priority"] = np.where(data["month"] < 11, True, False)

data.sort_values(
    ["year", "fips", "month_priority", "month", "day"], inplace=True
)
data.drop("month_priority", axis=1, inplace=True)
data.head(25)

,fips,month,day,temperature_min,precipitation,humidity_relative,wind_gust,wind_speed,wind_u_component,wind_v_component,solar_radiation_downward,vapor_pressure_deficit,skin_reservoir_content,soil_water_vol_layer1,soil_water_vol_layer2,soil_water_vol_layer3,images,year,yield_bu_per_acre
0,17001,11,1,273.936458,0.187208,71.783333,7.191125,3.372792,-2.079917,2.556042,444.233333,0.222625,4.768372e-07,0.310633,0.317448,0.258163,images/17001-2017-11-01,2018,197.8
1,17001,11,15,281.599000,5.130583,66.950000,8.537250,4.711750,2.010292,0.684500,2356.704167,0.475833,2.998769e-04,0.411875,0.311898,0.264373,images/17001-2017-11-15,2018,197.8
2,17001,12,1,270.082875,0.000000,54.183333,3.181625,2.111375,0.135792,1.666417,2724.837500,0.460125,5.270541e-06,0.313241,0.322834,0.293687,images/17001-2017-12-01,2018,197.8
3,17001,12,15,268.693333,0.196958,69.087500,5.463792,3.034458,2.569542,0.937000,1957.187500,0.215583,3.866851e-07,0.308099,0.315572,0.295845,images/17001-2017-12-15,2018,197.8
2300,17001,1,1,245.964750,0.000292,68.516667,6.681750,8.900583,6.154292,-6.210375,2779.654167,0.036583,1.524886e-07,0.295455,0.298550,0.295560,images/17001-2018-01-01,2018,197.8
2301,17001,1,15,259.711208,3.069542,79.375000,10.654000,7.629750,4.853792,-0.511083,2340.725000,0.063917,8.203089e-07,0.366524,0.354492,0.301671,images/17001-2018-01-15,2018,197.8
2302,17001,2,1,266.122750,0.000667,58.933333,8.360000,5.538167,3.720167,-3.772458,3289.820833,0.216667,1.456589e-07,0.345252,0.352277,0.326153,images/17001-2018-02-01,2018,197.8
2303,17001,2,15,278.780125,1.607292,90.712500,5.790583,3.133458,0.791167,1.074125,1149.287500,0.131958,2.816185e-04,0.391620,0.355211,0.329126,images/17001-2018-02-15,2018,197.8
2304,17001,3,1,277.554917,5.299417,84.329167,8.421125,5.573625,2.801000,-4.140708,3445.050000,0.196417,2.918119e-04,0.412618,0.403548,0.399821,images/17001-2018-03-01,2018,197.8
2305,17001,3,15,275.964875,0.000042,43.437500,5.680458,2.870417,0.963125,-0.001708,5511.870833,0.720917,5.314748e-07,0.359458,0.363917,0.385174,images/17001-2018-03-15,2018,197.8


## Обработка `NaN` значений

### 1. `fips`, `year`, `yield_bu_per_acre`

- Мы сделали `left join`. Не для каждого `year`, `fips` мы будем иметь значение `yield_bu_per_acre`. Нужно от них избавиться, учитывая условие ниже;
- для `year`, `fips` в текущего года, может не быть значения `yield_bu_per_acre` прошлого года, но для *ноябрь-сентябрь* предыдущего года нам нужны значения текущего.

In [9]:
# избавимся от значений, для которых нет данных в y.
# Я как будто олимпиадные задачи решаю
data = data[~((data["month"] < 11) & (data["yield_bu_per_acre"].isna()))]
data.head(3)

,fips,month,day,temperature_min,precipitation,humidity_relative,wind_gust,wind_speed,wind_u_component,wind_v_component,solar_radiation_downward,vapor_pressure_deficit,skin_reservoir_content,soil_water_vol_layer1,soil_water_vol_layer2,soil_water_vol_layer3,images,year,yield_bu_per_acre
0,17001,11,1,273.936458,0.187208,71.783333,7.191125,3.372792,-2.079917,2.556042,444.233333,0.222625,4.768372e-07,0.310633,0.317448,0.258163,images/17001-2017-11-01,2018,197.8
1,17001,11,15,281.599000,5.130583,66.950000,8.537250,4.711750,2.010292,0.684500,2356.704167,0.475833,2.998769e-04,0.411875,0.311898,0.264373,images/17001-2017-11-15,2018,197.8
2,17001,12,1,270.082875,0.000000,54.183333,3.181625,2.111375,0.135792,1.666417,2724.837500,0.460125,5.270541e-06,0.313241,0.322834,0.293687,images/17001-2017-12-01,2018,197.8


In [10]:
# Значение отсортированы по ["year", "fips", "month_priority", "month", "day"]
# Значит ноябрь-декабрь идут до январь-август
data.loc[:, ["year", "yield_bu_per_acre"]] = data[
    ["year", "yield_bu_per_acre"]
].bfill()

In [11]:
data[data.isna().any(axis=1)]

,fips,month,day,temperature_min,precipitation,humidity_relative,wind_gust,wind_speed,wind_u_component,wind_v_component,solar_radiation_downward,vapor_pressure_deficit,skin_reservoir_content,soil_water_vol_layer1,soil_water_vol_layer2,soil_water_vol_layer3,images,year,yield_bu_per_acre


Остались такие, для которых не было данных за *январь-август*

In [12]:
data = data[data.groupby(["year", "fips"])["month"].transform("nunique") == 12]

## Удаляем *сентябрь-октябрь*

*Сентябрь-октябрь* - время сбора урожая. Прогнозирование урожайности в этот период не имеет смысла. К тому же мы не знаем точного месяца сбора.

In [13]:
data = data[(data["month"] != 9) & (data["month"] != 10)]

## Разделяем на обучающую и тестовую выборку

In [14]:
mask = data["year"] == data["year"].max()
data_train = data[~mask]
data_test = data[mask]

X_train = data_train.drop("yield_bu_per_acre", axis=1)
y_train = data_train["yield_bu_per_acre"]
X_test = data_test.drop("yield_bu_per_acre", axis=1)
y_test = data_test["yield_bu_per_acre"]

In [15]:
print(f"X_train: {X_train.shape[0]}")
print(f"y_train: {y_train.shape[0]}")
print(f"X_test: {X_test.shape[0]}")
print(f"y_test: {y_test.shape[0]}")
print(f"(test size)/(data size): {X_test.shape[0] / data.shape[0]}")

X_train: 37060
y_train: 37060
X_test: 9800
y_test: 9800
(test size)/(data size): 0.20913358941527957


In [16]:
columns_order = ["year", "fips", "month", "day"] + np.sort(
    X_train.columns.drop(["year", "fips", "month", "day"])
).tolist()
X_train = X_train[columns_order]
X_test = X_test[columns_order]

In [17]:
X_train.head(25)

,year,fips,month,day,humidity_relative,images,precipitation,skin_reservoir_content,soil_water_vol_layer1,soil_water_vol_layer2,soil_water_vol_layer3,solar_radiation_downward,temperature_min,vapor_pressure_deficit,wind_gust,wind_speed,wind_u_component,wind_v_component
0,2018,17001,11,1,71.783333,images/17001-2017-11-01,0.187208,4.768372e-07,0.310633,0.317448,0.258163,444.233333,273.936458,0.222625,7.191125,3.372792,-2.079917,2.556042
1,2018,17001,11,15,66.950000,images/17001-2017-11-15,5.130583,2.998769e-04,0.411875,0.311898,0.264373,2356.704167,281.599000,0.475833,8.537250,4.711750,2.010292,0.684500
2,2018,17001,12,1,54.183333,images/17001-2017-12-01,0.000000,5.270541e-06,0.313241,0.322834,0.293687,2724.837500,270.082875,0.460125,3.181625,2.111375,0.135792,1.666417
3,2018,17001,12,15,69.087500,images/17001-2017-12-15,0.196958,3.866851e-07,0.308099,0.315572,0.295845,1957.187500,268.693333,0.215583,5.463792,3.034458,2.569542,0.937000
2300,2018,17001,1,1,68.516667,images/17001-2018-01-01,0.000292,1.524886e-07,0.295455,0.298550,0.295560,2779.654167,245.964750,0.036583,6.681750,8.900583,6.154292,-6.210375
2301,2018,17001,1,15,79.375000,images/17001-2018-01-15,3.069542,8.203089e-07,0.366524,0.354492,0.301671,2340.725000,259.711208,0.063917,10.654000,7.629750,4.853792,-0.511083
2302,2018,17001,2,1,58.933333,images/17001-2018-02-01,0.000667,1.456589e-07,0.345252,0.352277,0.326153,3289.820833,266.122750,0.216667,8.360000,5.538167,3.720167,-3.772458
2303,2018,17001,2,15,90.712500,images/17001-2018-02-15,1.607292,2.816185e-04,0.391620,0.355211,0.329126,1149.287500,278.780125,0.131958,5.790583,3.133458,0.791167,1.074125
2304,2018,17001,3,1,84.329167,images/17001-2018-03-01,5.299417,2.918119e-04,0.412618,0.403548,0.399821,3445.050000,277.554917,0.196417,8.421125,5.573625,2.801000,-4.140708
2305,2018,17001,3,15,43.437500,images/17001-2018-03-15,0.000042,5.314748e-07,0.359458,0.363917,0.385174,5511.870833,275.964875,0.720917,5.680458,2.870417,0.963125,-0.001708


## Проверка

- Соединяем X и y
- Проверяем, что для пары (year, fips) существует только одно единственное значение `yield_bu_per_acre`
- Проверяем, имеют ли объекты одинаковую длину (12 месяцев по 2 дня = 20)
- Разделяем X и y

In [18]:
concat = lambda X, y: pd.concat([X, y], axis=1)

data_train = concat(X_train, y_train)
data_test = concat(X_test, y_test)

for X_y in [data_train, data_test]:
    X_y_grouped = X_y.groupby(["year", "fips"])
    assert (X_y_grouped["yield_bu_per_acre"].nunique() == 1).all()
    assert (X_y_grouped["yield_bu_per_acre"].count() == 20).all()

## Сохраняем

In [19]:
if not PATH_PROCESSED.exists():
    PATH_PROCESSED.mkdir()
X_train.to_csv(PATH_PROCESSED / "X_train.csv", index=False)
y_train.to_csv(PATH_PROCESSED / "y_train.csv", index=False)
X_test.to_csv(PATH_PROCESSED / "X_test.csv", index=False)
y_test.to_csv(PATH_PROCESSED / "y_test.csv", index=False)

In [20]:
print(
    "X_train",
    X_train_size := X_train[["year", "fips"]].drop_duplicates().shape[0],
)
print(
    "X_test",
    X_test_size := X_test[["year", "fips"]].drop_duplicates().shape[0],
)
print("X_test/X", X_test_size / (X_test_size + X_train_size))

X_train 1853
X_test 490
X_test/X 0.20913358941527957
